In [ ]:
import pandas as pd
import numpy as np 
import csv as csv
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.lines import Line2D
import alphashape
import random
import json
import math

from scipy.stats import poisson
from sklearn.cluster import DBSCAN

import geopandas as gpd
import pyproj
from pyproj import Proj, transform, CRS
from shapely.geometry import Polygon, Point, MultiPolygon
from shapely.ops import unary_union

import re
from tqdm import tqdm

# Transformation der Bevölkerungsdaten

In [ ]:
file_path = './Einwohner_Data/Bevoelkerung100M.csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1', delimiter=';')

In [ ]:
# Liste der Merkmale, die gefiltert werden sollen
list_of_merkmale = [' INSGESAMT', 'ALTER_KURZ', 'GESCHLECHT', 'ALTER_10JG']

# Filtern des DataFrames df nach den angegebenen Merkmalen
df_einwohner = df[df['Merkmal'].isin(list_of_merkmale)]

# Erstellen eines leeren DataFrames für die Bezeichnungen
df_bezeichnungen = pd.DataFrame()

# Liste der Spaltennamen für die Bezeichnungen
bezeichnungen = ['Merkmal', 'Auspraegung_Text', 'Auspraegung_Code']

# Extrahieren der Bezeichnungsinformationen aus df_einwohner
df_bezeichnungen = df_einwohner[bezeichnungen]

# Entfernen von Duplikaten
df_bezeichnungen.drop_duplicates(inplace=True)

# Sortieren der Bezeichnungen nach 'Merkmal' und 'Auspraegung_Code'
df_bezeichnungen.sort_values(by=['Merkmal', 'Auspraegung_Code'], inplace=True)

# Zurücksetzen des Indexes
df_bezeichnungen.reset_index(drop=True, inplace=True)

# Speichern der Bezeichnungen als CSV-Datei
df_bezeichnungen.to_csv('./Einwohner_Data/Bezeichnungen.csv', index=False)

# Erstellen einer neuen Spalte 'Attribute' durch Kombination von 'Merkmal' und 'Auspraegung_Code'
df_einwohner['Attribute'] = df_einwohner['Merkmal'] + '_' + df_einwohner['Auspraegung_Code'].astype(str)

# Entfernen nicht benötigter Spalten
df_einwohner.drop(columns=['Merkmal', 'Auspraegung_Code', 'Auspraegung_Text', 'Gitter_ID_100m', 'Anzahl_q'], inplace=True)

In [ ]:
pivot_df = df_einwohner.pivot(index=['Gitter_ID_100m_neu'], columns=['Attribute'], values='Anzahl').reset_index()
pivot_df.columns.name = None
pivot_df = pivot_df.fillna(0)
pivot_df.rename(columns={' INSGESAMT_0': 'INSGESAMT_0'}, inplace=True)
pivot_df.to_csv('./Einwohner_Data/Einwohner_Cleaned.csv')

In [ ]:
# Einlesen der bereinigten Einwohnerdaten aus einer CSV-Datei
df_geo = pd.read_csv('./Einwohner_Data/Einwohner_Cleaned.csv')

# Extrahieren von Informationen aus den Gitter-IDs mithilfe von regulären Ausdrücken
pattern = re.compile(r'N(\d+)E(\d+)')
df_geo[['origin_n', 'origin_e']] = df_geo['Gitter_ID_100m_neu'].str.extract(pattern).astype(int)

# Setzen fester Werte für crs_code und resolution
df_geo['crs_code'] = 3035
df_geo['resolution'] = 100

# Erstellen von Geometrien (Quadraten) für jedes Gitter
df_geo['geometry'] = [Polygon([(e, n), (e + 100, n), (e + 100, n - 100), (e, n - 100)]) for e, n in zip(df_geo['origin_e'], df_geo['origin_n'])]

# Speichern der Geodaten als CSV-Datei
df_geo.to_csv('./GeoDaten/GeoDataFrame_Bevoelkerung.csv')

In [ ]:
gdf = gpd.GeoDataFrame(df_geo, geometry='geometry', crs=CRS(f"EPSG:{df_geo['crs_code'].iloc[0]}"))
gdf.to_file('./GeoDaten/Deutschland_Raster.gpkg', driver='GPKG')

In [ ]:
vmin, vmax = 0, 20
# Plotte das GeoDataFrame mit Farbhervorhebung der 'INSGESAMT'-Werte und angepasster Skala
gdf.plot(column='INSGESAMT_0', cmap='viridis', legend=True, figsize=(10, 10), vmin=vmin, vmax=vmax)
plt.title('Farbliche Hervorhebung der INSGESAMT-Werte')
plt.show()

# Filtern der Bevölkerungsdaten nach den NUTS-Grenzen

In [ ]:
gdf = gpd.read_file('./99_Old/GeoDaten/Deutschland_Raster_Bevoelkerung.gpkg')

In [ ]:
# Einlesen der Geodaten der Bundesländer aus einer GeoPackage-Datei
geo_bundesländer = gpd.read_file('./99_Old/GeoDaten/DE_NUTS5000.gpkg')

# Filtern der Geodaten, um nur die Einträge für Würzburg zu erhalten
geo_würzburg = geo_bundesländer[geo_bundesländer['NUTS_NAME'].str.contains('Würzburg')]

# Konvertieren der Koordinatenreferenzsystem (CRS) von geo_würzburg, um es dem CRS von gdf anzupassen
geo_würzburg = geo_würzburg.to_crs(gdf.crs)

# Speichern der gefilterten und konvertierten Geodaten als GeoPackage-Datei
geo_würzburg.to_file('./Wuerzburg_Data/geo_wuerzburg.gpkg', driver='GPKG')

# Erstellen einer Maske, um die überlappenden Geometrien zwischen gdf und geo_würzburg zu finden
mask_overlapping = gdf.geometry.intersects(geo_würzburg.unary_union)

# Filtern von gdf, um nur die überlappenden Geometrien zu behalten
gdf_würzburg = gdf[mask_overlapping]

# Speichern der gefilterten Geodaten als GeoPackage-Datei
gdf_würzburg.to_file('./Wuerzburg_Data/Raster_Wuerzburg.gpkg', driver='GPKG')

In [ ]:
# Einlesen der Geodaten der Bundesländer aus einer GeoPackage-Datei
geo_bundesländer = gpd.read_file('./GeoDaten/DE_NUTS5000.gpkg')

# Filtern der Geodaten, um nur die Einträge für Frankfurt am Main zu erhalten
geo_frankfurt = geo_bundesländer[geo_bundesländer['NUTS_NAME'].str.contains('Frankfurt am Main')]

# Konvertieren des Koordinatenreferenzsystems (CRS) von geo_frankfurt zu EPSG:3035
geo_frankfurt = geo_frankfurt.to_crs('3035')

# Speichern der gefilterten und konvertierten Geodaten als GeoPackage-Datei
geo_frankfurt.to_file('./Frankfurt_Data/geo_frankfurt.gpkg', driver='GPKG')

# Erstellen einer Maske, um die überlappenden Geometrien zwischen gdf und geo_frankfurt zu finden
mask_overlapping = gdf.geometry.intersects(geo_frankfurt.unary_union)

# Filtern von gdf, um nur die überlappenden Geometrien zu behalten
gdf_frankfurt = gdf[mask_overlapping]

# Speichern der gefilterten Geodaten als GeoPackage-Datei
gdf_frankfurt.to_file('./Frankfurt_Data/Raster_Frankfurt.gpkg', driver='GPKG')

In [ ]:
# Einlesen der Geodaten der Bundesländer aus einer GeoPackage-Datei
geo_bundesländer = gpd.read_file('./GeoDaten/DE_NUTS5000.gpkg')

# Filtern der Geodaten, um nur die Einträge für Donnersbergkreis zu erhalten
geo_donner = geo_bundesländer[geo_bundesländer['NUTS_NAME'].str.contains('Donnersbergkreis')]

# Konvertieren des Koordinatenreferenzsystems (CRS) von geo_donner zu EPSG:3035
geo_donner = geo_donner.to_crs('3035')

# Speichern der gefilterten und konvertierten Geodaten als GeoPackage-Datei
geo_donner.to_file('./Donner_Data/geo_donner.gpkg', driver='GPKG')

# Erstellen einer Maske, um die überlappenden Geometrien zwischen gdf und geo_donner zu finden
mask_overlapping = gdf.geometry.intersects(geo_donner.unary_union)

# Filtern von gdf, um nur die überlappenden Geometrien zu behalten
gdf_donner = gdf[mask_overlapping]

# Speichern der gefilterten Geodaten als GeoPackage-Datei
gdf_donner.to_file('./Donner_Data/Raster_Donner.gpkg', driver='GPKG')

# Zuweisung der Bevölkerungsdaten auf die Gebäudedaten

In [ ]:
df_buildings = pd.read_csv('./Donner_Data/Donner-Buildings.csv')
gdf_würzburg = gpd.read_file('./Donner_Data/Raster_Donner.gpkg')

## Gebäude einem Raster zuordnen

In [ ]:
def find_nearest_polygon_id(point, polygons):
    """
    Findet die ID des nächstgelegenen Polygons zu einem gegebenen Punkt.

    Args:
        point (GeoDataFrame): Ein GeoDataFrame, der den Punkt enthält.
        polygons (GeoDataFrame): Ein GeoDataFrame, das die Polygone enthält.

    Returns:
        int: Die ID des nächstgelegenen Polygons.
    """
    # Berechnung der Distanz zwischen dem Punkt und allen Polygonen
    distances = polygons.distance(point.geometry)
    
    # Sortieren der Distanzen und Abrufen des Indexes (ID) des nächstgelegenen Polygons
    nearest_polygon_id = distances.sort_values().index[0]
    
    return nearest_polygon_id


In [ ]:
# Erstellen eines GeoDataFrame für die Gebäude mit den Koordinaten und dem CRS EPSG:4326
gdf_buildings = gpd.GeoDataFrame(df_buildings, geometry=gpd.points_from_xy(df_buildings['lon'], df_buildings['lat']), crs=CRS("EPSG:4326"))

# Umprojizieren des GeoDataFrames der Gebäude in das CRS von gdf_würzburg
gdf_buildings = gdf_buildings.to_crs(gdf_würzburg.crs)

# Initialisieren der 'Raster_ID' Spalte mit -1
df_buildings['Raster_ID'] = -1

# Iterieren über jede Zeile im GeoDataFrame der Gebäude
for index, building in gdf_buildings.iterrows():
    # Erstellen einer Maske, um zu prüfen, ob das Gebäude mit einem Polygon in gdf_würzburg überlappt
    mask_overlapping = gdf_würzburg.intersects(building['geometry'])
    
    if any(mask_overlapping):
        # Wenn das Gebäude mit einem Polygon überlappt, setze die 'Raster_ID' auf die Index des ersten überlappenden Polygons
        df_buildings.loc[index, 'Raster_ID'] = gdf_würzburg[mask_overlapping].index[0]
    else:
        # Wenn das Gebäude mit keinem Polygon überlappt, finde das nächstgelegene Polygon und setze die 'Raster_ID'
        df_buildings.loc[index, 'Raster_ID'] = find_nearest_polygon_id(building, gdf_würzburg)

# Erstellen eines neuen GeoDataFrames mit den Gebäuden und den zugewiesenen Raster-IDs
gdf_buildings_with_raster = gpd.GeoDataFrame(df_buildings, geometry=gdf_buildings['geometry'], crs=gdf_würzburg.crs)

In [ ]:
# Zählen der eindeutigen Werte in der Spalte 'Raster_ID'
value_counts = gdf_buildings_with_raster['Raster_ID'].value_counts()

# Sortieren der Ergebnisse nach den Werten in absteigender Reihenfolge
sorted_value_counts = value_counts.sort_values(ascending=False)

# Ausgabe der sortierten Ergebnisse
print("Sortierte Wertezählungen:")
print(sorted_value_counts)

## Jedem Gebäude die Bevölkerung zuweisen

In [ ]:
# Initialisieren neuer Spalten in gdf_buildings_with_raster
gdf_buildings_with_raster['Einwohner'] = 0
gdf_buildings_with_raster['Alter'] = {} 
gdf_buildings_with_raster['Geschlecht'] = {} 

# Sammeln eindeutiger Raster-IDs
unique_raster_indices = gdf_buildings_with_raster['Raster_ID'].unique()

# Durchlaufe alle eindeutigen Raster-IDs
for raster_index in unique_raster_indices:
    # Anzahl der Einwohner aus dem Raster extrahieren
    einwohner_aus_raster = gdf_würzburg.at[raster_index, 'INSGESAMT_0']

    # Altersverteilung aus dem Raster extrahieren
    alter_dict_raster = {
        '0-10':  int(gdf_würzburg.at[raster_index, 'ALTER_10JG_1']),
        '10-20': int(gdf_würzburg.at[raster_index, 'ALTER_10JG_2']),
        '20-30': int(gdf_würzburg.at[raster_index, 'ALTER_10JG_3']),
        '30-40': int(gdf_würzburg.at[raster_index, 'ALTER_10JG_4']), 
        '40-50': int(gdf_würzburg.at[raster_index, 'ALTER_10JG_5']), 
        '50-60': int(gdf_würzburg.at[raster_index, 'ALTER_10JG_6']),
        '60-70': int(gdf_würzburg.at[raster_index, 'ALTER_10JG_7']), 
        '70-80': int(gdf_würzburg.at[raster_index, 'ALTER_10JG_8']),
        '80+':   int(gdf_würzburg.at[raster_index, 'ALTER_10JG_9'])
    }

    # Geschlechtsverteilung aus dem Raster extrahieren
    geschlecht_dict_raster = {
        'maennlich': gdf_würzburg.at[raster_index, 'GESCHLECHT_1'],
        'weiblich':  gdf_würzburg.at[raster_index, 'GESCHLECHT_2']
    }

    # Durchlaufe alle Gebäude im aktuellen Raster
    for index, building in gdf_buildings_with_raster[gdf_buildings_with_raster['Raster_ID'] == raster_index].iterrows():
        
        # Initialisieren von Dictionaries für Alters- und Geschlechtsverteilungen im Gebäude
        alter_dict_geb = {
            '0-10': 0,
            '10-20': 0,
            '20-30': 0,
            '30-40': 0, 
            '40-50': 0, 
            '50-60': 0,
            '60-70': 0, 
            '70-80': 0,
            '80+': 0
        }

        geschlecht_dict_geb = {
            'maennlich': 0,
            'weiblich': 0
        }

        # Zuweisung einer zufälligen Anzahl von Einwohnern zum Gebäude
        if index == gdf_buildings_with_raster[gdf_buildings_with_raster['Raster_ID'] == raster_index].index.max():
            random_einwohner = einwohner_aus_raster
        else:
            random_einwohner = np.random.choice(int(einwohner_aus_raster))

        random_einwohner = int(random_einwohner)
        alter_count = random_einwohner
        geschlecht_count = random_einwohner

        # Zufällige Zuweisung von Alter
        for key in np.random.permutation(list(alter_dict_raster.keys())):
            if alter_count == 0:
                break
            if alter_dict_raster[key] > 0:
                if alter_count <= alter_dict_raster[key]:
                    alter_dict_geb[key] = alter_count
                    alter_dict_raster[key] -= alter_count
                    alter_count = 0
                else:
                    alter_dict_geb[key] = alter_dict_raster[key]
                    alter_count -= alter_dict_raster[key]
                    alter_dict_raster[key] = 0

        # Falls noch ein Restwert vorhanden ist, zufällig einem Altersbereich zuweisen
        if alter_count != 0:
            alter_dict_geb[random.choice(list(alter_dict_geb.keys()))] += alter_count

        # Zufällige Zuweisung von Geschlecht
        for key in np.random.permutation(list(geschlecht_dict_raster.keys())):
            if geschlecht_count == 0:
                break
            if geschlecht_dict_raster[key] > 0:
                if geschlecht_count <= geschlecht_dict_raster[key]:
                    geschlecht_dict_geb[key] = geschlecht_count
                    geschlecht_dict_raster[key] -= geschlecht_count
                    geschlecht_count = 0
                else:
                    geschlecht_dict_geb[key] = geschlecht_dict_raster[key]
                    geschlecht_count -= geschlecht_dict_raster[key]
                    geschlecht_dict_raster[key] = 0

        # Falls noch ein Restwert vorhanden ist, zufällig einem Geschlecht zuweisen
        if geschlecht_count != 0:
            geschlecht_dict_geb[random.choice(list(geschlecht_dict_geb.keys()))] += geschlecht_count

        # Aktualisieren der Gebäudeinformationen im GeoDataFrame
        gdf_buildings_with_raster.at[index, 'Einwohner'] = random_einwohner
        gdf_buildings_with_raster.loc[index, 'Alter'] = [alter_dict_geb]
        gdf_buildings_with_raster.loc[index, 'Geschlecht'] = [geschlecht_dict_geb]
        einwohner_aus_raster -= random_einwohner

In [ ]:
gdf_buildings_with_raster['Alter'] = gdf_buildings_with_raster['Alter'].apply(json.dumps)
gdf_buildings_with_raster['Geschlecht'] = gdf_buildings_with_raster['Geschlecht'].apply(json.dumps)

gdf_buildings_with_raster.to_file('./Donner_Data/Buildings_Raster_Demographie.gpkg', driver='GPKG')

# Zuweisung der nächsten Apotheke zu den Gebäuden

In [ ]:
# Zu entfernende Spalten definieren
columns_to_drop = ['name', 'building', 'addr:street', 'addr:housenumber', 'addr:postcode']

# Einlesen der Gebäudedaten mit Raster- und Demographiedaten aus einer GeoPackage-Datei
bevölkerungs_gdf = gpd.read_file('./Donner_Data/Buildings_Raster_Demographie.gpkg')

# Umwandeln der Spalten 'Alter' und 'Geschlecht' in Dictionaries
bevölkerungs_gdf['Alter'] = bevölkerungs_gdf['Alter'].apply(json.loads)
bevölkerungs_gdf['Geschlecht'] = bevölkerungs_gdf['Geschlecht'].apply(json.loads)

# Entfernen der definierten Spalten aus dem DataFrame
bevölkerungs_gdf.drop(columns=columns_to_drop, inplace=True)

# Einlesen der Apothekendaten aus einer CSV-Datei
pharmacy_df = pd.read_csv('./Donner_Data/Donner-Apotheken.csv')

# Erstellen eines GeoDataFrame für die Apotheken mit den Koordinaten und dem CRS EPSG:4326
pharmacy_gdf = gpd.GeoDataFrame(pharmacy_df, geometry=gpd.points_from_xy(pharmacy_df['lon'], pharmacy_df['lat']), crs=CRS("EPSG:4326"))

# Umprojizieren des GeoDataFrames der Apotheken in das CRS von bevölkerungs_gdf
pharmacy_gdf = pharmacy_gdf.to_crs(bevölkerungs_gdf.crs)

In [ ]:
def find_nearest_point_with_cKDTree(lon, lat, points_df):
    """
    Findet den nächstgelegenen Punkt in einem DataFrame basierend auf Längen- und Breitengrad.
    
    Parameter:
    lon (float): Längengrad des Referenzpunkts.
    lat (float): Breitengrad des Referenzpunkts.
    points_df (DataFrame): DataFrame, das die Punkte mit den Spalten 'lon' und 'lat' enthält.

    Rückgabe:
    Series: Die Zeile des nächstgelegenen Punkts im DataFrame.
    """
    # Extrahiere die Koordinaten aus dem DataFrame als NumPy-Array
    coordinates = np.array(points_df[['lon', 'lat']])
    
    # Erstelle ein NumPy-Array für den Referenzpunkt
    reference_point = np.array([lon, lat])
    
    # Berechne die paarweisen quadrierten Distanzen (schneller für cKDTree)
    distances = np.sum((coordinates - reference_point)**2, axis=1)
    
    # Finde den Index der minimalen Distanz
    idx = np.argmin(distances)

    # Gib die Zeile des nächstgelegenen Punkts zurück
    return points_df.iloc[idx]  # Verwende iloc für schnellere Integer-Indexierung

In [ ]:
def calculate_distance_to_nearest_pharmacy(row):
    """
    Findet die nächstgelegene Apotheke zu einem gegebenen Punkt und gibt die ID der nächstgelegenen Apotheke zurück.

    Parameter:
    row (Series): Eine Zeile des DataFrame, die die Koordinaten 'lon' und 'lat' des Punkts enthält.

    Rückgabe:
    str: Die ID der nächstgelegenen Apotheke.
    """
    # Finde die nächstgelegene Apotheke zu den Koordinaten des gegebenen Punkts
    nearest_pharmacy = find_nearest_point_with_cKDTree(row.lon, row.lat, pharmacy_gdf)
    
    # Gib die ID der nächstgelegenen Apotheke zurück
    return nearest_pharmacy['id']

In [ ]:
# Initialisieren der Spalte 'assigned_pharmacy' in bevölkerungs_gdf mit 0
bevölkerungs_gdf['assigned_pharmacy'] = 0 

# Aktivieren der Fortschrittsanzeige für Pandas-Operationen
tqdm.pandas()

# Durchlaufen aller Zeilen im DataFrame bevölkerungs_gdf
for index, row in tqdm(bevölkerungs_gdf.iterrows(), total=len(bevölkerungs_gdf)):
    # Berechnen der nächstgelegenen Apotheke für die aktuelle Zeile
    pharmacy = calculate_distance_to_nearest_pharmacy(row)
    # Zuweisen der ID der nächstgelegenen Apotheke zur Spalte 'assigned_pharmacy'
    bevölkerungs_gdf.loc[index, 'assigned_pharmacy'] = pharmacy

In [ ]:
bevölkerungs_gdf['Alter'] = bevölkerungs_gdf['Alter'].apply(json.dumps)
bevölkerungs_gdf['Geschlecht'] = bevölkerungs_gdf['Geschlecht'].apply(json.dumps)
bevölkerungs_gdf.to_file('./Donner_Data/pharmacy_assigned.gpkg', driver='GPKG')

# Nachfrage-Wahrscheinlichkeitsverteilung

In [ ]:
# Frequenz von Apothekenbesuchen nach Altersverteilung
# Die Kategorien der Besuchshäufigkeit sind:
# "Mehrmals in der Woche, Etwa einmal in der Woche, Zwei- bis dreimal im Monat, Einmal im Monat, Etwa einmal im Vierteljahr, Seltener, Nie, so gut wie nie"

# Dictionary mit der Verteilung der Apothekenbesuche nach Altersgruppen
apotheken_besuch = {
    '14-19': [0.002, 0.005, 0.023, 0.053, 0.18, 0.305, 0.432],  # Altersgruppe 14-19 Jahre
    '20-29': [0.003, 0.009, 0.052, 0.131, 0.282, 0.326, 0.197],  # Altersgruppe 20-29 Jahre
    '30-39': [0.006, 0.013, 0.098, 0.198, 0.287, 0.275, 0.123],  # Altersgruppe 30-39 Jahre
    '40-49': [0.009, 0.014, 0.105, 0.21, 0.297, 0.258, 0.106],   # Altersgruppe 40-49 Jahre
    '50-59': [0.008, 0.019, 0.116, 0.252, 0.308, 0.209, 0.087],  # Altersgruppe 50-59 Jahre
    '60-69': [0.005, 0.037, 0.173, 0.277, 0.278, 0.159, 0.071],  # Altersgruppe 60-69 Jahre
    '70+':   [0.011, 0.065, 0.252, 0.336, 0.196, 0.064, 0.075]   # Altersgruppe 70+ Jahre
}

In [ ]:
# Initialisieren des Dictionaries für die Altersverteilung
alter_verteilung = {
    "10-20": 0,
    "20-30": 0,
    "30-40": 0,
    "40-50": 0,
    "50-60": 0,
    "60-70": 0,
    "70-80": 0
}

# Extrahieren der Wahrscheinlichkeiten aus dem apotheken_besuch Dictionary und Umwandeln in ein NumPy Array
prob_werte = list(apotheken_besuch.values())
prob_arrays = np.stack(prob_werte)

# Extrahieren der Schlüssel (Altersgruppen) aus dem alter_verteilung Dictionary
keys = list(alter_verteilung.keys())

# Initialisieren der Liste für die Wahrscheinlichkeiten
probabilities = []

# Berechnung der täglichen Besuchswahrscheinlichkeiten basierend auf den monatlichen Verteilungen
for i in range(5):
    for j in range(7):  
        if i == 0:  # Mehrmals in der Woche
            alter_verteilung[keys[j]] += prob_arrays[j][i] / 3.85714285714
        elif i == 1:  # Etwa einmal in der Woche
            alter_verteilung[keys[j]] += prob_arrays[j][i] / 7
        elif i == 2:  # Zwei- bis dreimal im Monat
            alter_verteilung[keys[j]] += prob_arrays[j][i] / 12
        elif i == 3:  # Einmal im Monat
            alter_verteilung[keys[j]] += prob_arrays[j][i] / 30
        elif i == 4:  # Etwa einmal im Vierteljahr
            alter_verteilung[keys[j]] += prob_arrays[j][i] / 90

# Anpassung der Verteilung für die Altersgruppe "10-20"
alter_verteilung["10-20"] = alter_verteilung["10-20"] / 2

# Ausgabe der berechneten Altersverteilung
print(alter_verteilung)

# Clustering der Raster

In [ ]:
gdf_city = gpd.read_file('./99_Old/Frankfurt/Raster_frankfurt.gpkg')
geo_gdf = gpd.read_file('./Frankfurt_Data/geo_frankfurt.gpkg')

In [ ]:
# Extrahieren der Polygon-Koordinaten als Features für das Clustering
# Erstellen eines NumPy-Arrays mit den Koordinaten der Zentroiden der Polygone
X = np.array(gdf_city.geometry.apply(lambda polygon: [polygon.centroid.x, polygon.centroid.y]).tolist())

# DBSCAN-Clustering durchführen
# Initialisieren des DBSCAN-Algorithmus mit den Parametern eps und min_samples
dbscan = DBSCAN(eps=100, min_samples=4)  # Anpassen von eps und min_samples je nach Bedarf

# Anpassen des DBSCAN-Modells an die Daten
dbscan.fit(X)

# Fügen Sie die Cluster-Zuordnung als neue Spalte zum GeoDataFrame hinzu
# Die Cluster-Zuordnungen (labels_) werden als neue Spalte 'cluster' zum GeoDataFrame hinzugefügt
gdf_city['cluster'] = dbscan.labels_

In [ ]:
# # Gruppieren nach Clustern und Polygone in MultiPolygone umwandeln
# multi_polygons = gdf_city.groupby('cluster')['geometry'].apply(lambda x: MultiPolygon(list(x)))
# # Ein neues GeoDataFrame erstellen mit den MultiPolygons
# gdf_multi = gpd.GeoDataFrame(multi_polygons, geometry='geometry').reset_index()

In [ ]:
# Funktion, um einen Teil einer Colormap zu extrahieren
def truncate_colormap(cmap, minval=0.5, maxval=1.0, n=100):
    new_cmap = plt.cm.colors.LinearSegmentedColormap.from_list(
        f'trunc({cmap.name},{minval:.2f},{maxval:.2f})', 
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

In [ ]:
# Verwende den helleren Teil der cividis-Colormap
cmap = truncate_colormap(plt.get_cmap('viridis'), 0.5, 1.0)

# Berechne die 10 größten Cluster ohne Cluster -1
top_clusters = gdf_city[gdf_city['cluster'] != -1]['cluster'].value_counts().nlargest(10).index

# Anzahl der Polygone im Cluster -1
outlier_count = len(gdf_city[gdf_city['cluster'] == -1])

# Plotten der Boundary
fig, ax = plt.subplots(figsize=(10, 10))
geo_gdf.boundary.plot(ax=ax, color='gray', linewidth=0.5)

# Plotten der Polygone mit Cluster-Färbung
gdf_city[gdf_city['cluster'] != -1].plot(ax=ax, column='cluster', cmap=cmap, legend=False)
gdf_city[gdf_city['cluster'] == -1].plot(ax=ax, color='purple', legend=False)  # Lila für Cluster -1

# for cluster_id in gdf_multi[gdf_multi['cluster']!= -1].cluster:
#     cluster_data = gdf_city[gdf_city['cluster'] == cluster_id]
#     # Kombiniere alle Geometrien des Clusters zu einer einzigen Geometrie
#     combined_geom = cluster_data.unary_union
#     # Berechne die konvexe Hülle der kombinierten Geometrie
#     if isinstance(combined_geom, (MultiPolygon, Polygon)):
#         outer_boundary = combined_geom.convex_hull
#         gpd.GeoSeries(outer_boundary).boundary.plot(ax=ax, color='lightgray', linewidth=0.4)
#     else:
#         for geom in combined_geom.geoms:
#             gpd.GeoSeries(geom.convex_hull).boundary.plot(ax=ax, color='lightgray', linewidth=0.4)


# Zentroiden und Anzahl der Polygone der 10 größten Cluster anzeigen
for cluster_id in top_clusters:
    cluster_data = gdf_city[gdf_city['cluster'] == cluster_id]
    centroid = cluster_data.geometry.unary_union.centroid
    polygon_count = len(cluster_data)
    ax.plot(centroid.x, centroid.y, '^', color='red', markersize=6)
    ax.text(centroid.x + 1800, centroid.y, str(polygon_count), fontsize=12, ha='center', va='center', color='black', fontweight='bold')  # Versetzte Position

# Hinzufügen eines Farbbalkens
norm = plt.Normalize(vmin=gdf_city['cluster'].min(), vmax=gdf_city['cluster'].max())
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, shrink=0.65)  # Colormap kleiner machen
cbar.set_label('ID der Cluster')
# Text unterhalb der Farblegende hinzufügen
plt.text(1.2, 0, f'Anzahl der Ausreiser: {outlier_count}', ha='right', va='center', transform=ax.transAxes, fontsize=10)


# Add scale bar
scalebar = ScaleBar(1, location='lower left', label='Maßstab')
ax.add_artist(scalebar)

plt.axis('off')
plt.show()

In [ ]:
# Alle eindeutigen Cluster-Werte und ihre Anzahl der Polygone ausgeben
cluster_counts = gdf_city['cluster'].value_counts().sort_index()
print("Cluster-Werte und ihre Anzahl der Polygone:")
for cluster, count in cluster_counts.items():
    print(f"Cluster {cluster}: {count} Polygone")

In [ ]:
# Plotten der Rauschpunkte (Clusterlabel -1)
noise_points = gdf_city[gdf_city['cluster'] == -1]
ax = noise_points.boundary.plot(color='red', alpha=0.5, figsize=(5, 5))
plt.title('Noise')
plt.xlabel('X-Achse')
plt.ylabel('Y-Achse')
plt.show()

In [ ]:
column_to_sum = 'INSGESAMT_0'
sum_of_column = noise_points[column_to_sum].sum()

print('Anzahl der Raster, die als Noise erkannt wurden: {}'.format(len(noise_points)))
print("Sum of column '{}' in noise points: {}".format(column_to_sum, sum_of_column))

In [ ]:
# Gruppierung nach dem Cluster und Aggregierung der Punkte in jedem Cluster
cluster_summary = gdf_city.groupby('cluster').agg(
    Gitter_ID_100m_neu=('Gitter_ID_100m_neu', 'first'),  # Erster Wert von Gitter_ID_100m_neu im Cluster
    count=('cluster', 'size'),  # Anzahl der Polygone in jedem Cluster
    sum_INSGESAMT_0=('INSGESAMT_0', 'sum'),  # Summe der Gesamtbevölkerung im Cluster
    sum_ALTER_10JG_1=('ALTER_10JG_1', 'sum'),  # Summe der Altersgruppe 0-10 Jahre im Cluster
    sum_ALTER_10JG_2=('ALTER_10JG_2', 'sum'),  # Summe der Altersgruppe 10-20 Jahre im Cluster
    sum_ALTER_10JG_3=('ALTER_10JG_3', 'sum'),  # Summe der Altersgruppe 20-30 Jahre im Cluster
    sum_ALTER_10JG_4=('ALTER_10JG_4', 'sum'),  # Summe der Altersgruppe 30-40 Jahre im Cluster
    sum_ALTER_10JG_5=('ALTER_10JG_5', 'sum'),  # Summe der Altersgruppe 40-50 Jahre im Cluster
    sum_ALTER_10JG_6=('ALTER_10JG_6', 'sum'),  # Summe der Altersgruppe 50-60 Jahre im Cluster
    sum_ALTER_10JG_7=('ALTER_10JG_7', 'sum'),  # Summe der Altersgruppe 60-70 Jahre im Cluster
    sum_ALTER_10JG_8=('ALTER_10JG_8', 'sum'),  # Summe der Altersgruppe 70-80 Jahre im Cluster
    sum_ALTER_10JG_9=('ALTER_10JG_9', 'sum'),  # Summe der Altersgruppe 80+ Jahre im Cluster
    sum_GESCHLECHT_1=('GESCHLECHT_1', 'sum'),  # Summe der männlichen Bevölkerung im Cluster
    sum_GESCHLECHT_2=('GESCHLECHT_2', 'sum'),  # Summe der weiblichen Bevölkerung im Cluster
    crs_code=('crs_code', 'first'),  # Erster Wert von crs_code im Cluster
    resolution=('resolution', 'first'),  # Erster Wert von resolution im Cluster
    origin_n=('origin_n', 'first'),  # Erster Wert von origin_n im Cluster
    origin_e=('origin_e', 'first'),  # Erster Wert von origin_e im Cluster
    geometry=('geometry', lambda x: unary_union(x))  # Vereinigung der Polygone in jedem Cluster
)

# Umwandeln des aggregierten DataFrames in ein GeoDataFrame
# Definieren eines neuen GeoDataFrames mit der aggregierten Geometrie und den Eigenschaften
cluster_summary_gdf = gpd.GeoDataFrame(cluster_summary, geometry='geometry', crs=gdf_city.crs)

# Zurücksetzen des Index, um den Cluster-Wert als Spalte beizubehalten
cluster_summary_gdf = cluster_summary_gdf.reset_index(drop=False)

In [ ]:
# Funktion zur Berechnung der Entfernung zwischen zwei Punkten
def distance(point1, point2):
    return point1.distance(point2)

# Berechnung der durchschnittlichen Entfernung von jedem Polygon-Centroid zu seinen Ecken
average_distances = []

# Iterieren über jede Zeile im GeoDataFrame
for idx, row in cluster_summary_gdf.iterrows():
    centroid = row['geometry'].centroid  # Berechnung des Zentroids des Polygons
    polygon = row['geometry']  # Abrufen der Polygongeometrie

    if isinstance(polygon, MultiPolygon):
        average_distance = 0  # Setzen der durchschnittlichen Entfernung auf 0 für Multipolygone
    else:
        # Extrahieren der Koordinaten der Polygon-Ecken
        polygon_corners = polygon.exterior.coords[:-1]  # Letzter Punkt ist der gleiche wie der erste
        
        # Berechnung der Entfernungen und Speichern der Ergebnisse
        distances = [distance(centroid, Point(coord)) for coord in polygon_corners]
        average_distance = sum(distances) / len(distances)
        
    average_distances.append(average_distance)

# Hinzufügen der durchschnittlichen Entfernungen als neue Spalte zum GeoDataFrame
cluster_summary_gdf['average_distance'] = average_distances

In [ ]:
# Plot der Raster mit Polygone farblich darstellen
ax = cluster_summary_gdf.plot(cmap='viridis', legend=True, figsize=(5, 5))

# Hervorhebung der Rastergrenzen
cluster_summary_gdf.boundary.plot(ax=ax, color='black')

# Titel hinzufügen
plt.title('Bevölkerungsraster des Landkreis Würzburg')

# Achsenbeschriftungen hinzufügen
plt.xlabel('Breitengradkoordinate im CSR3035 Format')
plt.ylabel('Längengradkoordinate im CSR3035 Format')

plt.show()

In [ ]:
# Definiere die Projektionen
in_proj = Proj(init='epsg:3035')  # CSR3035 (Eingangsprojektion)
out_proj = Proj(init='epsg:4326')  # WGS84 (Ausgangsprojektion, Längen- und Breitengrade)

# Funktion zum Umwandeln von CSR3035-Koordinaten in Längen- und Breitengrade
def csr3035_to_latlon(polygon):
    point = polygon.centroid  # Berechne den Zentroiden des Polygons
    x_csr3035 = point.x  # X-Koordinate des Zentroiden
    y_csr3035 = point.y  # Y-Koordinate des Zentroiden
    lon, lat = transform(in_proj, out_proj, x_csr3035, y_csr3035)  # Transformiere die Koordinaten in Längen- und Breitengrade
    return lon, lat  # Gebe die transformierten Koordinaten zurück

# Wende die Funktion auf die Geometry-Spalte an und erstelle neue Spalten für Längen- und Breitengrade
cluster_summary_gdf['lon'], cluster_summary_gdf['lat'] = zip(*cluster_summary_gdf['geometry'].apply(lambda point: csr3035_to_latlon(point)))

In [ ]:
# Identifikation der Spalten, die mit 'sum_ALTER_10JG_' beginnen
age_group_columns = [col for col in cluster_summary_gdf.columns if col.startswith('sum_ALTER_10JG_')]

# Definition der neuen Altersgruppenstruktur
new_age_groups = {
    '0-10': 0,
    '10-20': 3,
    '20-30': 0,
    '30-40': 0,
    '40-50': 6,
    '50-60': 8,
    '60-70': 0,
    '70-80': 9,
    '80+': 0
}

# Erstellung einer Liste der neuen Altersgruppen
age_groups_list = list(new_age_groups.keys())

# Funktion zur Extraktion der Altersgruppendaten aus dem DataFrame
def get_age_group_data(cluster_summary_gdf):
    age_group_data = {}

    # Iteration durch die ersten 9 Altersgruppen
    for count in range(9):
        age_group = age_groups_list[count]  # Extraktion der Altersgruppe aus der Liste
        population = cluster_summary_gdf[age_group_columns[count]]  # Extraktion der Populationsdaten für die Altersgruppe
        age_group_data[age_group] = population  # Speicherung der Populationsdaten in einem Dictionary
    return age_group_data

# Extraktion und Zusammenführung der Altersgruppendaten für jede Zeile im DataFrame
age_group_data_per_cluster = cluster_summary_gdf.apply(get_age_group_data, axis=1)

# Hinzufügen der extrahierten Altersgruppendaten als neue Spalte
cluster_summary_gdf['Alter'] = age_group_data_per_cluster

# Entfernung der ursprünglichen 'ALTER_10JG_' Spalten
cluster_summary_gdf.drop(columns=age_group_columns, inplace=True)

In [ ]:
# Polygon Geopandas speichern
cluster_summary_gdf.to_file('./Frankfurt_Data/cluster_frankfurt.gpkg', driver='GPKG')